In [1]:
import imports
import numpy as np
from sklearn.cluster import KMeans
import sklearn.datasets as skdatasets
import sklearn.metrics as skmetrics 
from metrics import accuracy
from initialisations import onoda2012 as onoda
from sklearn.decomposition import FastICA, PCA

In [2]:
dataset = skdatasets.load_iris()
data = dataset.data
target = dataset.target

K = 3

whiten_data=False

### TODO:

 * whiten/centre/standardise data?

In [3]:
def calc_distance(row, component):
    def mag(vector):
        return np.linalg.norm(vector)
    
    return np.dot(component, row) / (mag(component) * mag(row))

## PCA

nb. ``KernelPCA`` and ``IncrementalPCA`` exist too.

In [7]:
# PCA(n_components=None, copy=True, whiten=False, svd_solver=’auto’, 
#        tol=0.0, iterated_power=’auto’, random_state=None)

pca = PCA(n_components=K, whiten=whiten_data)
pca.fit(data)
print(pca.components_) 

C = []

for component in pca.components_:
    distances = [calc_distance(x, component) for x in data]
    C.append(data[np.argmin(distances)])

centroids = np.array(C)

est = KMeans(n_clusters=K, n_init=1, init=centroids)
est.fit(data)
print(est.cluster_centers_)


[[ 0.36138659 -0.08452251  0.85667061  0.3582892 ]
 [ 0.65658877  0.73016143 -0.17337266 -0.07548102]
 [-0.58202985  0.59791083  0.07623608  0.54583143]]
[[5.006      3.428      1.462      0.246     ]
 [6.85384615 3.07692308 5.71538462 2.05384615]
 [5.88360656 2.74098361 4.38852459 1.43442623]]


### PCA Metrics

In [8]:
acc = accuracy.score(target, est.labels_)
ari = skmetrics.adjusted_rand_score(target, est.labels_)

print("Accuracy Score:", acc)
print("Adjusted Rand Index:", ari)
print("Inertia:", est.inertia_)

Accuracy Score: 0.8866666666666667
Adjusted Rand Index: 0.7163421126838475
Inertia: 78.8556658259773


## ICA

In [ ]:
# Compute ICA
ica = FastICA(n_components=K) # return 3x4
#ica = FastICA() # returns 4x4 - completely different numbers
S_ = ica.fit_transform(data)  # Reconstruct signals
#A_ = ica.mixing_  # Get estimated mixing matrix

print(ica.components_) # The unmixing matrix

print(len(S_))
#print(A_)

# We can `prove` that the ICA model applies by reverting the unmixing.
#assert np.allclose(X, np.dot(S_, A_.T) + ica.mean_)

In [ ]:
# For comparison, compute PCA
#pca = PCA(n_components=K)
#H = pca.fit_transform(data)  # Reconstruct signals based on orthogonal components
#print(H)